In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


# 使用Torchtext预处理自定义文本数据集

**作者**: [Anupam Sharma](https://anp-scp.github.io/)

本教程演示了如何在不使用内置数据集的情况下使用torchtext处理自定义数据集。在本教程中，我们将预处理一个数据集，以便进一步用于训练序列到序列模型进行机器翻译（类似于本教程中的[序列到序列学习与神经网络](https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb)），但不使用torchtext的旧版本。

在本教程中，我们将学习如何：

- 读取数据集
- 对句子进行分词
- 对句子应用转换
- 进行桶式批处理

假设我们需要准备一个数据集来训练一个能够进行英语到德语翻译的模型。我们将使用[Tatoeba项目](https://tatoeba.org/en)提供的一个制表符分隔的德语 - 英语句对数据集，可以从[此链接](https://www.manythings.org/anki/deu-eng.zip)下载。

其他语言的句对可以在[此链接](https://www.manythings.org/anki/)找到。


## 设置

首先，下载数据集，解压缩zip文件，并记录文件`deu.txt`的路径。

确保已安装以下软件包：

- [Torchdata 0.6.0](https://pytorch.org/data/beta/index.html)（[安装说明](https://github.com/pytorch/data)）
- [Torchtext 0.15.0](https://pytorch.org/text/stable/index.html)（[安装说明](https://github.com/pytorch/text)）
- [Spacy](https://spacy.io/usage)

这里，我们使用`Spacy`进行文本分词。简而言之，分词是将句子转换为单词列表的过程。Spacy是用于各种自然语言处理（NLP）任务的Python包。

按照以下方式从Spacy下载英语和德语模型：

```
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm

```


导入模块:


In [10]:
import torchdata.datapipes as dp
import torchtext.transforms as T
import spacy
from torchtext.vocab import build_vocab_from_iterator
eng = spacy.load("en_core_web_sm")  # Load the English model to tokenize English text
de = spacy.load("de_core_news_sm")  # Load the German model to tokenize German text
print(de.path)

/opt/anaconda3/envs/learning-in-general/lib/python3.10/site-packages/de_core_news_sm/de_core_news_sm-3.7.0


读取数据集



In [13]:
FILE_PATH = '../../data/deu-eng/deu.txt'
data_pipe = dp.iter.IterableWrapper([FILE_PATH])
data_pipe = dp.iter.FileOpener(data_pipe, mode='rb')
data_pipe = data_pipe.parse_csv(skip_lines=0, delimiter='\t', as_tuple=True)

在上面的代码块中，我们正在进行以下操作：

1. 在第2行，我们创建了一个文件名的可迭代对象
2. 在第3行，我们将可迭代对象传递给`FileOpener`，然后
以读取模式打开文件
3. 在第4行，我们调用一个函数来解析文件，该函数
再次返回一个表示每行的元组的可迭代对象，其中每个元组表示制表符分隔的文件的每行

DataPipes可以被视为类似于数据集对象的东西，我们可以在其上执行各种操作。
有关DataPipes的更多详细信息，请查看[此教程](https://pytorch.org/data/beta/dp_tutorial.html)。

我们可以验证可迭代对象是否具有如下所示的一对句子：

In [14]:
for sample in data_pipe:
    print(sample)
    break

('Go.', 'Geh.', 'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8597805 (Roujin)')


请注意，我们还附带了license。我们将编写一个小函数来删除license：



In [15]:
def removeAttribution(row):
    """
    Function to keep the first two elements in a tuple
    """
    return row[:2]
data_pipe = data_pipe.map(removeAttribution)

在上述代码块中，第6行的map函数可以用于对data_pipe的每个元素应用某个函数。现在，我们可以验证data_pipe只包含句子对。


In [16]:
for sample in data_pipe:
    print(sample)
    break

('Go.', 'Geh.')


现在，让我们定义一些函数来执行分词操作：


In [17]:
def engTokenize(text):
    """
    Tokenize an English text and return a list of tokens
    """
    return [token.text for token in eng.tokenizer(text)]

def deTokenize(text):
    """
    Tokenize a German text and return a list of tokens
    """
    return [token.text for token in de.tokenizer(text)]

上述函数接受一个文本，并返回一个单词列表，如下所示：



In [18]:
print(engTokenize("Have a good day!!!"))
print(deTokenize("Haben Sie einen guten Tag!!!"))

['Have', 'a', 'good', 'day', '!', '!', '!']
['Haben', 'Sie', 'einen', 'guten', 'Tag', '!', '!', '!']


## 构建词汇表

让我们将一个英文句子作为源句子，一个德文句子作为目标句子。

词汇表可以被看作是数据集中唯一单词的集合。
我们现在将为源句子和目标句子构建词汇表。

让我们定义一个函数，从迭代器中的元组元素中获取标记。


In [19]:
def getTokens(data_iter, place):
    """
    Function to yield tokens from an iterator. Since, our iterator contains
    tuple of sentences (source and target), `place` parameters defines for which
    index to return the tokens for. `place=0` for source and `place=1` for target
    """
    for english, german in data_iter:
        if place == 0:
            yield engTokenize(english)
        else:
            yield deTokenize(german)

现在，我们将为源语言建立词汇表：


In [20]:
source_vocab = build_vocab_from_iterator(
    getTokens(data_pipe,0),
    min_freq=2,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
source_vocab.set_default_index(source_vocab['<unk>'])

上面的代码从迭代器中构建词汇表。在上面的代码块中：

- 在第2行，我们使用`place=0`调用`getTokens()`函数，因为我们需要源句子的词汇表。
- 在第3行，我们设置`min_freq=2`。这意味着该函数将跳过出现少于2次的单词。
- 在第4行，我们指定了一些特殊的标记：
    - `<sos>`表示句子的开头
    - `<eos>`表示句子的结尾
    - `<unk>`表示未知单词。一个未知单词的例子是因为`min_freq=2`而被跳过的单词。
    - `<pad>`是填充标记。在训练过程中，我们通常以批次进行训练。在一个批次中，可能有不同长度的句子。因此，我们用`<pad>`标记填充较短的句子，使得批次中所有序列的长度相等。
- 在第5行，我们设置`special_first=True`。这意味着`<pad>`的索引为0，`<sos>`的索引为1，`<eos>`的索引为2，`<unk>`的索引为3。
- 在第7行，我们将默认索引设置为`<unk>`的索引。这意味着如果某个单词不在词汇表中，我们将使用`<unk>`代替该未知单词。

类似地，我们将为目标句子构建词汇表：

In [21]:
target_vocab = build_vocab_from_iterator(
    getTokens(data_pipe,1),
    min_freq=2,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
target_vocab.set_default_index(target_vocab['<unk>'])

请注意，上面的示例展示了如何向我们的词汇表中添加特殊标记。特殊标记可能会根据需求而变化。

现在，我们可以验证特殊标记是否放置在开头，然后是其他单词。
在下面的代码中，`source_vocab.get_itos()` 返回一个基于词汇表索引的标记列表。


In [22]:
print(source_vocab.get_itos()[:9])

['<pad>', '<sos>', '<eos>', '<unk>', '.', 'I', 'Tom', 'to', 'you']


## 使用词汇表对句子进行数字化

在构建词汇表之后，我们需要将句子转换为相应的索引。
让我们为此定义一些函数：

In [23]:
def getTransform(vocab):
    """
    Create transforms based on given vocabulary. The returned transform is applied to sequence
    of tokens.
    """
    text_tranform = T.Sequential(
        ## converts the sentences to indices based on given vocabulary
        T.VocabTransform(vocab=vocab),
        ## Add <sos> at beginning of each sentence. 1 because the index for <sos> in vocabulary is
        # 1 as seen in previous section
        T.AddToken(1, begin=True),
        ## Add <eos> at beginning of each sentence. 2 because the index for <eos> in vocabulary is
        # 2 as seen in previous section
        T.AddToken(2, begin=False)
    )
    return text_tranform

现在，让我们看看如何使用上述函数。该函数返回一个`Transforms`对象，我们将在我们的句子上使用它。让我们取一个随机的句子并检查转换的工作原理。


In [24]:
temp_list = list(data_pipe)
some_sentence = temp_list[798][0]
print("Some sentence=", end="")
print(some_sentence)
transformed_sentence = getTransform(source_vocab)(engTokenize(some_sentence))
print("Transformed sentence=", end="")
print(transformed_sentence)
index_to_string = source_vocab.get_itos()
for index in transformed_sentence:
    print(index_to_string[index], end=" ")

Some sentence=I fainted.
Transformed sentence=[1, 5, 2897, 4, 2]
<sos> I fainted . <eos> 

在上面的代码中：

- 在第2行，我们从`data_pipe`在第1行创建的列表中获取源句子
- 在第5行，我们基于源词汇获取一个转换，并将其应用于一个标记化的句子。请注意，转换接受单词列表而不是句子。
- 在第8行，我们获取索引到字符串的映射，然后使用它来获取转换后的句子

现在我们将使用DataPipe函数将转换应用于所有的句子。
让我们为此定义一些更多的函数。

In [25]:
def applyTransform(sequence_pair):
    """
    Apply transforms to sequence of tokens in a sequence pair
    """

    return (
        getTransform(source_vocab)(engTokenize(sequence_pair[0])),
        getTransform(target_vocab)(deTokenize(sequence_pair[1]))
    )
data_pipe = data_pipe.map(applyTransform) ## Apply the function to each element in the iterator
temp_list = list(data_pipe)
print(temp_list[0])

([1, 616, 4, 2], [1, 739, 4, 2])


## 制作批次（使用bucket batch）

通常，我们会以批次的方式训练模型。在处理序列到序列模型时，建议保持批次中序列的长度相似。为此，我们将使用`data_pipe`中的`bucketbatch`函数。

让我们定义一些将被`bucketbatch`函数使用的函数。


In [26]:
def sortBucket(bucket):
    """
    Function to sort a given bucket. Here, we want to sort based on the length of
    source and target sequence.
    """
    return sorted(bucket, key=lambda x: (len(x[0]), len(x[1])))

接下来，我们会使用 `bucketbatch` 函数:



In [27]:
data_pipe = data_pipe.bucketbatch(
    batch_size = 4, batch_num=5,  bucket_num=1,
    use_in_batch_shuffle=False, sort_key=sortBucket
)

在上述代码块中：

- 我们保持批大小为4。
- `batch_num`是要在一个桶中保持的批的数量
- `bucket_num`是要在一个池中保持的桶的数量，用于洗牌
- `sort_key`指定了一个函数，该函数接受一个桶并对其进行排序

现在，让我们将源句子的批次称为`X`，将目标句子的批次称为`y`。
通常，在训练模型时，我们对一个批次的`X`进行预测，并将结果与`y`进行比较。
但是，在我们的`data_pipe`中，一个批次的形式是`[(X_1,y_1), (X_2,y_2), (X_3,y_3), (X_4,y_4)]`：


In [28]:
print(list(data_pipe)[0])

[([1, 1066, 4, 2], [1, 1383, 4, 2]), ([1, 2989, 4, 2], [1, 6030, 1616, 24, 2]), ([1, 3, 194, 2], [1, 740, 2445, 24, 2]), ([1, 1670, 194, 2], [1, 1210, 3, 24, 2])]


所以，我们现在将它们转换成形式：((X_1,X_2,X_3,X_4), (y_1,y_2,y_3,y_4))。
为此，我们将编写一个小函数：

In [29]:
def separateSourceTarget(sequence_pairs):
    """
    input of form: `[(X_1,y_1), (X_2,y_2), (X_3,y_3), (X_4,y_4)]`
    output of form: `((X_1,X_2,X_3,X_4), (y_1,y_2,y_3,y_4))`
    """
    sources,targets = zip(*sequence_pairs)
    return sources,targets

## Apply the function to each element in the iterator
data_pipe = data_pipe.map(separateSourceTarget)
print(list(data_pipe)[0])

(([1, 2014, 68, 4, 2], [1, 616, 98, 4, 2], [1, 1042, 21, 4, 2], [1, 6860, 23, 194, 2]), ([1, 2228, 28, 24, 2], [1, 739, 106, 24, 2], [1, 380, 12, 32, 24, 2], [1, 7, 1048, 383, 24, 2]))


现在，我们已经得到了所需的数据。

## 填充

在构建词汇表时我们讨论过，我们需要将一个批次中的较短句子进行填充，以使批次中的所有序列长度相等。我们可以按照以下方式进行填充：

In [30]:
def applyPadding(pair_of_sequences):
    """
    Convert sequences to tensors and apply padding
    """
    return (T.ToTensor(0)(list(pair_of_sequences[0])), T.ToTensor(0)(list(pair_of_sequences[1])))
## `T.ToTensor(0)` returns a transform that converts the sequence to `torch.tensor` and also applies
# padding. Here, `0` is passed to the constructor to specify the index of the `<pad>` token in the
# vocabulary.
data_pipe = data_pipe.map(applyPadding)

现在，我们可以使用索引到字符串映射来查看序列在使用标记而不是索引时的样子：


In [33]:
source_index_to_string = source_vocab.get_itos()
target_index_to_string = target_vocab.get_itos()

def showSomeTransformedSentences(data_pipe):
    """
    Function to show how the sentences look like after applying all transforms.
    Here we try to print actual words instead of corresponding index
    """
    for sources,targets in data_pipe:
        if sources[0][-1] != 0:
            continue # Just to visualize padding of shorter sentences
        for i in range(4):
            source = ""
            for token in sources[i]:
                source += " " + source_index_to_string[token]
            target = ""
            for token in targets[i]:
                target += " " + target_index_to_string[token]
            print(f"Source: {source}")
            print(f"Target: {target}")
        break

showSomeTransformedSentences(data_pipe)

Source:  <sos> Freeze ! <eos> <pad>
Target:  <sos> Stehenbleiben ! <eos> <pad>
Source:  <sos> <unk> ! <eos> <pad>
Target:  <sos> Zum Wohl ! <eos>
Source:  <sos> Freeze ! <eos> <pad>
Target:  <sos> Keine Bewegung ! <eos>
Source:  <sos> Got it ! <eos>
Target:  <sos> Verstanden ! <eos> <pad>


在上面的输出中，我们可以观察到较短的句子被填充了`<pad>`。现在，我们可以在编写训练函数时使用`data_pipe`。

本教程的一些部分受到了[这篇文章](https://medium.com/@bitdribble/migrate-torchtext-to-the-new-0-9-0-api-1ff1472b5d71)的启发。